In [8]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import os

data_folder = r"/workspaces/Projects/Cashew_Board/data"
file_on_folder = os.listdir(data_folder)[0]

def android_to_hex(color_code):
    hex_code = "#{:06x}".format(color_code & 0xffffff)
    return hex_code

df = pd.read_csv(fr"{data_folder}/{file_on_folder}", engine = "pyarrow", keep_default_na = False)
df["color"] = df["color"].apply(android_to_hex) # Transform color code to hex
df['amount'] = df['amount'].abs() # Transform negative values to positive
df["date"] = pd.to_datetime(df["date"].dt.strftime("%m-%d-%Y")) # Format date to keep only month-day-year


df

,account,amount,currency,title,note,date,income,type,category name,subcategory name,color,icon,emoji,budget,objective
0,Main,12.0,USD,,,2024-11-21,False,null,Taxi,,#ffee58,taxi(1).png,,,
1,Main,8.0,USD,,,2024-11-21,False,null,Taxi,,#ffee58,taxi(1).png,,,
2,Main,8.0,USD,,,2024-11-21,False,null,Taxi,,#ffee58,taxi(1).png,,,
3,Main,8.0,USD,,,2024-11-21,False,null,Taxi,,#ffee58,taxi(1).png,,,
4,Main,20.0,USD,,Proton Duo wi,2024-11-20,False,null,Subscription,,#ff7043,subscription.png,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,Main,1.0,USD,,,2024-01-03,False,null,Groceries,,#4caf50,groceries.png,,,
1284,Main,10.0,USD,,,2024-01-02,True,null,Tip,,#66bb6a,dollar-coin.png,,,
1285,Main,16.0,USD,,,2024-01-02,False,null,Taxi,,#ffee58,taxi(1).png,,,
1286,Main,14.0,USD,,,2024-01-02,False,null,Taxi,,#ffee58,taxi(1).png,,,


In [9]:
def plot_category_map(df: pd.DataFrame, category : str):
    
    df = df[df["category name"] == category]
    df["subcategory name"] = df["subcategory name"].apply(lambda row: "None" if row == "" else row)
    df = df.groupby(by = ["date", "color", "category name", "subcategory name"]).agg({"amount": "sum"}).reset_index()
    
    fig = go.Figure(layout = go.Layout(height = 500, width = 750, title = "Expenses Map by Category", template = "plotly_dark"))
    fig.add_trace(go.Scatter(x = df["date"],
                             x0= df["subcategory name"],
                             y = df["amount"],
                             hovertext = df["subcategory name"],
                             hoverlabel = dict(font_size = 15),
                             hovertemplate = "$%{y} On %{x} <extra></extra>: Subcategory: %{hovertext}",
                             mode = "markers",
                             marker_color = df["color"],
                             marker_size = df["amount"] / 5,
                             showlegend = False))
    
    return fig

plot_category_map(df, "Taxi")

/tmp/ipykernel_11450/4170608298.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

